# Features engineering

In [1]:
# Libraries and parameters
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import math

In [2]:
# Import data
data = pd.read_csv("../raw_data/final_raw_data.csv", index_col="Unnamed: 0")

In [3]:
# #Creating hitter stats up to at bat
data['hitter_previous_stats_szn'] = data.groupby("hitter_id")["y_target"].cumsum() / (data.groupby('hitter_id').cumcount() + 1)
data['rolling_1ab'] = data.groupby("hitter_id")["y_target"].transform(lambda x: x.shift().rolling(1, min_periods=1).mean())
data['rolling_3ab'] = data.groupby("hitter_id")["y_target"].transform(lambda x: x.shift().rolling(3, min_periods=1).mean())
data['rolling_10ab'] = data.groupby("hitter_id")["y_target"].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())

In [4]:
# Calculate stats of pitcher before he pitchs
data['pitcher_previous_stats_szn'] = data.groupby("pitcher_id")["y_target"].cumsum() / (data.groupby('pitcher_id').cumcount() + 1)
data['rolling_1pitch'] = data.groupby("pitcher_id")["y_target"].transform(lambda x: x.shift().rolling(1, min_periods=1).mean())
data['rolling_3pitch'] = data.groupby("pitcher_id")["y_target"].transform(lambda x: x.shift().rolling(3, min_periods=1).mean())
data['rolling_10pitch'] = data.groupby("pitcher_id")["y_target"].transform(lambda x: x.shift().rolling(10, min_periods=1).mean())

In [5]:
#Create handed matchups
data["handed_matchup"] = data["hitter_hand"] + data["pitcher_hand"]
data.head()

,id,game_id,inning,side,hitter_id,hitter_hand,pitcher_id,pitcher_hand,temp_f,weather_condition,humidity,wind_speed_mph,at_bat_end_time,pitch_location_zone,pitch_type_code,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,y_target,pitch_class,day_night,home_team_id,away_team_id,attendance,stadium_id,hitter_player_name,hitter_position,hitter_primary_position,pitcher_player_name,pitcher_primary_position,stadium_capacity,stadium_stadium_type,stadium_lat,stadium_lon,away_stadium_lat,away_stadium_lon,hitter_previous_stats_szn,rolling_1ab,rolling_3ab,rolling_10ab,pitcher_previous_stats_szn,rolling_1pitch,rolling_3pitch,rolling_10pitch,handed_matchup
0,d43f7325-ca19-49ab-94f6-b92055c26dd5,f3f0ae8e-cb65-4e96-9530-c6f868738f09,1,T,041632a9-afb2-4ec3-b1de-9b0bbe33ab64,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,Sunny,32.0,9.0,2023-03-30 17:11:07+00:00,11.0,FF,94.4,4.0,0.0,0.0,1,1,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Wade Jr., LaMonte",IF,1B,"Cole, Gerrit",SP,47309.0,outdoor,40.828819,-73.926569,37.77842,-122.390621,1.0,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,LR
1,f7f6d34f-5e1e-4ca9-896a-1f026054cbf5,f3f0ae8e-cb65-4e96-9530-c6f868738f09,1,T,e5bdeb0e-38fc-4d30-8127-43d0d5b2864d,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,Sunny,32.0,9.0,2023-03-30 17:13:06+00:00,8.0,SL,86.8,5.0,4.0,0.0,0,0,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Conforto, Michael",OF,RF,"Cole, Gerrit",SP,47309.0,outdoor,40.828819,-73.926569,37.77842,-122.390621,0.0,NaN,NaN,NaN,0.500000,1.0,1.000000,1.000000,LR
2,faad7e49-b9a2-4359-85a4-695438b6885c,f3f0ae8e-cb65-4e96-9530-c6f868738f09,1,T,45bca5ce-4514-4a91-9410-e201c7b0052d,R,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,Sunny,32.0,9.0,2023-03-30 17:14:55+00:00,6.0,SL,89.8,5.0,9.0,1.0,0,0,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Flores, Wilmer",IF,1B,"Cole, Gerrit",SP,47309.0,outdoor,40.828819,-73.926569,37.77842,-122.390621,0.0,NaN,NaN,NaN,0.333333,0.0,0.500000,0.500000,RR
3,8f33e404-d2c3-427e-8a98-96ea1945285e,f3f0ae8e-cb65-4e96-9530-c6f868738f09,1,T,43a9d631-5673-4059-9b25-d59290bc32c3,L,eebc991a-23ea-4f1c-ba3b-37ff21ee1603,R,38.0,Sunny,32.0,9.0,2023-03-30 17:18:09+00:00,14.0,SL,89.1,7.0,14.0,2.0,0,1,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"Pederson, Joc",OF,LF,"Cole, Gerrit",SP,47309.0,outdoor,40.828819,-73.926569,37.77842,-122.390621,0.0,NaN,NaN,NaN,0.250000,0.0,0.333333,0.333333,LR
4,b2fbcf1d-862e-4102-bb4e-fbac3656031c,f3f0ae8e-cb65-4e96-9530-c6f868738f09,1,B,d1c07b01-a86d-4c17-ac38-8217e364c2cf,R,72248e3e-3b0f-4523-9ff6-833aa33f3b32,R,38.0,Sunny,32.0,9.0,2023-03-30 17:21:39+00:00,12.0,SL,85.9,4.0,0.0,0.0,0,1,D,a09ec676-f887-43dc-bbb3-cf4bbaee9a18,a7723160-10b7-4277-a309-d8dd95a8ae65,46172.0,706e9828-6687-4ac8-a409-3fb972e8bae9,"LeMahieu, David",IF,3B,"Webb, Logan",SP,47309.0,outdoor,40.828819,-73.926569,37.77842,-122.390621,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,RR


In [6]:
#total hitter and pitcher at bats prior to at bat
data["hitter_ab_count"] = data.groupby('hitter_id').cumcount()
data["pitcher_ab_count"] = data.groupby('pitcher_id').cumcount()
data["match_up_ab_count_delta"] = data["pitcher_ab_count"] - data["hitter_ab_count"] 

In [7]:
#Balls and strikes efficiencies, batters
data["reverse_pitch_class"] = data["pitch_class"].apply(lambda x: 0 if x == 1 else 1)

data["hitter_strikes_atb"] = (data.groupby("hitter_id")["reverse_pitch_class"].cumsum())
data["hitter_balls_atb"] = (data.groupby("hitter_id")["pitch_class"].cumsum())

data["hitter_strikes_only_results"] = data["y_target"] * data["reverse_pitch_class"]
data["hitter_balls_only_results"] = data["y_target"] * data["pitch_class"]

data["hitter_strikes_results"] = (data.groupby("hitter_id")["hitter_strikes_only_results"].cumsum())
data["hitter_balls_results"] = (data.groupby("hitter_id")["hitter_balls_only_results"].cumsum())

data["hitter_strikes_eff"] = data["hitter_strikes_results"] / data["hitter_strikes_atb"]
data["hitter_balls_eff"] = data["hitter_balls_results"] / data["hitter_balls_atb"]

#Balls and strikes efficiencies, pitchers

data["pitcher_total_atb"] = (data.groupby("pitcher_id").cumcount())
data["pitcher_strikes_spread"] = (data.groupby("pitcher_id")["reverse_pitch_class"].cumsum()) / data["pitcher_total_atb"]
data["pitcher_balls_spread"] = (data.groupby("pitcher_id")["pitch_class"].cumsum()) / data["pitcher_total_atb"]


#pitch speed stats
data["hitter_speeds_class1"] = data["pitch_speed_mph"] * data["y_target"]
data["hitter_success_speed"] = (data.groupby("hitter_id")["hitter_speeds_class1"].cumsum()) / (data.groupby("hitter_id")["y_target"].cumsum())

data["pitcher_speed"] = (data.groupby("pitcher_id")["pitch_speed_mph"].cumsum())/data["pitcher_total_atb"]


#dropping redundant columns to avoid multi-colinearity

data = data.drop(columns=["pitch_class", "reverse_pitch_class", "hitter_strikes_atb", "hitter_balls_atb", "hitter_strikes_only_results",
                         "hitter_balls_only_results", "hitter_strikes_results", "hitter_balls_results", "pitcher_total_atb", "hitter_speeds_class1", "pitch_type_code"])

In [10]:
#drop 0.004343% of data (first ten at bats for each player)
data = data.dropna(subset=["rolling_10ab", "rolling_10pitch", "hitter_strikes_eff", "hitter_balls_eff", "hitter_success_speed"])

In [12]:
data.isnull().sum()

id                                    0
game_id                               0
inning                                0
side                                  0
hitter_id                             0
hitter_hand                           0
pitcher_id                            0
pitcher_hand                          0
temp_f                                0
weather_condition                     0
humidity                              0
wind_speed_mph                        0
at_bat_end_time                       0
pitch_location_zone                   0
pitch_speed_mph                       0
pitch_count_at_bat                    0
pitcher_pitch_count_at_bat_start      0
outs_at_start                         0
y_target                              0
day_night                             0
home_team_id                          0
away_team_id                          0
attendance                          438
stadium_id                            0
hitter_player_name                    0


In [13]:
# data[data["hitter_id"] == "a8efa694-2d64-4ad3-bf39-7d4f9006be7f"].to_csv('../../archive/test_rolling_ab.csv')
# data[data["pitcher_id"] == "e27962b7-39b4-43b5-9e6f-0fe27dcf2ead"].to_csv('../../archive/test_rolling_pitch.csv')

In [14]:
data.to_csv("../preproc_data/data_with_new_features.csv")